In [0]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns; sns.set()
import pandas as pd
import random

#Regulatory Motifs & Hidden Messages in DNA

Coming fresh off our OriCalculator project, we saw the benefits of finding frequent strands and narrowing down windows for OriC. However towards the end we saw the weakness of our algorithm if a strand had a mutation or slight mismatch with our original, we would have a tough time finding it. This is where motif finding comes in. 
This is helpful in situations where you worry you may have mutated the subject in an experiment or conversely, implanting a mismatched motif into a strand that you now want to locate. It is also important because some motifs are responsible for certain actions in an organism, from awakening dormant TB spores to clock genes that control your circadian clock. Finding these can be imperative in different fields.


We'd love it if these motifs were conserved but depending on position, they can bind to a slightly mismatched string, and we want to find these to make sense of them.

Much like the OriCalculator, it should be noted that many of these algorithms could be streamlined with the use of numeric processing libraries like numpy/tensorflow and data manipulation ones like pandas. However to focus on teaching basic python fundamentals and not overwhelm you, we'll focus on purely what Python can offer.

In [0]:
sample_genome = 'ATCAATGATCAACGTAAGCTTCTAAGCATGATCAAGGTGCTCACACAGTTTATCCACAACCTGAGTGGATGACATCAAGATAGGTCGTTGTATCTCCTTCCTCTCGTACTCTCATGACCACGGAAAGATGATCAAGAGAGGATGATTTCTTGGCCATATCGCAATGAATACTTGTGACTTGTGCTTCCAATTGACATCTTCAGCGCCATATTGCGCTGGCCAAGGTGACGGAGCGGGATTACGAAAGCATGATCATGGCTGTTGTTCTGTTTATCTTGTTTTGACTGAGACTTGTTAGGATAGACGGTTTTTCATCACTGACTAGCCAAAGCCTTACTCTGCCTGACATCGACCGTAAATTGATAATGAATTTACATGCTTCCGCGACGATTTACCTCTTGATCATCGATCCGATTGAAGATCTTCAATTGTTAATTCTCTTGCCTCGACTCATAGCCATGATGAGCTCTTGATCATGTTTCCTTAACCCTCTATTTTTTACGGAAGAATGATCAAGCTGCTGCTCTTGATCATCGTTTC'
sample_dna_list = ['TCGGG', 'CCGGT', 'ACGGG', 'TTGGG']
#Arbitrary string and list for us to test our functions/pipeline on.

## DNA Sequence Manipulation and Matrices Creation

In [0]:
def nuc_counter(Motifs):
  '''
  
  Input: A list of DNA strands
  
  Output: A matrix containing the sum of occurrences of a nucleotide of each strand at every position. (See below for example explanation)
  
  '''
  count = {}
  kmer_len = len(Motifs[0])
  for letter in "ACGT":
      count[letter] = []
      for j in range(kmer_len):
          count[letter].append(1)
  for i in range(len(Motifs)):
      for j in range(kmer_len):
          symbol = Motifs[i][j]
          count[symbol][j] += 1 
  return count
  
  
def nuc_counter_profile(Motifs):
  '''
  
  Input: A list of DNA strands
  
  Output: A matrix containing the sum of occurrences of a nucleotide of each strand at every position divided by number of DNA strands. (See below for example explanation)
  
  '''
  profile = {}
  kmer_len = len(Motifs[0])
  for letter in "ACGT":
      profile[letter] = []
      for j in range(kmer_len):
          profile[letter].append(1)
  for i in range(len(Motifs)):
      for j in range(kmer_len):
          symbol = Motifs[i][j]
          profile[symbol][j] += 1
  for letter in "ACGT":
      for position in range(kmer_len):
          profile[letter][position] = profile[letter][position]/(len(Motifs)+4)
  return profile


def consensus_string(Motifs):
  '''
  
  Input: A list of DNA strands
  Output: The string comprised of the most frequent nucleotides at each position. (See below for example explanation)
  
  '''

  count = nuc_counter(Motifs)
  consensus = ""
  for i in range(len(Motifs[0])):
      cur_max = 0
      frequentletter = ""
      for letter in "ACGT":
          if count[letter][i] >= cur_max:
              cur_max = count[letter][i]
              frequentletter = letter
      consensus += frequentletter
  return consensus
 
  
def motif_scoring(Motifs):
  '''
  
  Input: A list of DNA strands
  Output: The string comprised of the most frequent nucleotides at each position. (See below for example explanation)
  
  '''
  score = 0
  consensus = consensus_string(Motifs)
  for i in range(len(Motifs)):
      score += HammingDistance(Motifs[i], consensus)
  return score 

  
def HammingDistance(p, q):
  '''
  Simple formula to calculate how many mismatches there ar between two strings (p and q)
  '''
  count = 0
  for i in range(len(p)):
      if p[i] != q[i]:
          count += 1
  return count

Given a set of DNA strings. We apply the above functions in a certain way to gain insight on them. They are general useful functions for changing motif data. For example given the following strings:

T C G G G

C C G G T

A C G G G

T T G G G

We turn these into a matrix. And these are what we will call our motifs.
Let's apply our functions on them. 



In [0]:
print(nuc_counter(sample_dna_list))
print(nuc_counter_profile(sample_dna_list))
print(consensus_string(sample_dna_list))
print(motif_scoring(sample_dna_list))

{'A': [2, 1, 1, 1, 1], 'C': [2, 4, 1, 1, 1], 'G': [1, 1, 5, 5, 4], 'T': [3, 2, 1, 1, 2]}
{'A': [0.25, 0.125, 0.125, 0.125, 0.125], 'C': [0.25, 0.5, 0.125, 0.125, 0.125], 'G': [0.125, 0.125, 0.625, 0.625, 0.5], 'T': [0.375, 0.25, 0.125, 0.125, 0.25]}
TCGGG
4


So let's examine what happens. The following is our motif matrix of intial DNA strands. 


T C G G G (Motifs)

C C G G T

A C G G G

T T G G G
_____________________
This consensus is the most frequent nucleotide in each column.

T C G G G (Consensus)
_____________________
The score is the amount of nucleotides that do not match the most frequent one. We usually want to minimize this score to get the most 'conserved' kmer.

2+1+0+0+1 = 4 (Score)

_____________________
The count matrix depicts how often each nucleotide occurs in each column. *Notice that there is a '1' added where there should be a zero. This is to avoid an entire string's probability being equaled to 0 even when sometimes the rest of its string may only differ by 1 spot.*


A: [2, 1, 1, 1, 1] (Motif Counts)

C: [2, 4, 1, 1, 1]

G: [1, 1, 5, 5, 4]

T': [3, 2, 1, 1, 2]
_____________________
The profile matrix divides each count by the number of nucleotides in each column, to give you a probability of 'generating' that nucleotide.


A: [0.25, 0.125, 0.125, 0.125, 0.125] (Motif Profile)

C: [0.25, 0.5, 0.125, 0.125, 0.125]

G: [0.125, 0.125, 0.625, 0.625, 0.5]

T: [0.375, 0.25, 0.125, 0.125, 0.25]






## Greedy algorithms in Motif Finding

In [0]:
def Pr(Kmer, profile):
  
  '''
  Input: A specific kmer strand and a pre-calculated profile matrix of probabilities.
  
  Output: The probability of getting that specific kmer based off the matrix.
  '''
  
  p = 1
  for i in range(len(Kmer)):
      p = p * profile[Kmer[i]][i]
  return p

  
def kmer_with_max_probability(sample_genome, t, profile):
  
  '''
  Input: A sample string, k for length of kmer you want to find, and a profile matrix of probabilities you are searching through.
  Output: Most probable kmer in a string (from the probabilities of a previously generated profile matrix)
  
  
  So for an example text/genome such as ACCGGTTGT, every possible say 3mer like ACC, CCG,CGG  etc. is evaluated vs the profile
  Profile being a 4xK(3 in this case) e.g: A: 0.3 0.4 0.2
                                           C: 0.2 0.2 0.3
                                           G: 0.4 0.3 0.1
                                           T: 0.3 0.3 0.5
  Probability is calculated (ACC = 0.3 * 0.2 * 0.3 for example)
  Max probability is recorded, along with its 3mer.
  And thus, the most probable 3mer in this specific profile i.e a 3 mer that was 
  most likely to be generated by a Profile among all Kmers in 'text'
                                           
  '''
  max_pr = -1
  for i in  range(len(sample_genome) -  t+1):
    kmer = sample_genome[i:i+t]
    current_pr = Pr(kmer, profile)
    if current_pr > max_pr:
      max_pr = current_pr
      max_pr_kmer = kmer

  return max_pr_kmer


def GreedyMotifSearch(Dna, k):
  
  """
  A culmination of all our functions so far. Given a DNA set of strings, we want to find a set of motifs that match each ohter
  closely. 
  To start with an input of example set of DNA strings, k = 3 and number of strings = 5:
  
  ACCGGTTGT
  GCTAGGTAC
  CCTAGTACG
  ACAACTTGT
  ATTACCTGT
  
  We take the first 3mers of each string as a basis to begin comparing future motifs against. So [ACC, GCT, CCT, ACA, ATT].
  We have an empty string named Motifs we want to begin using as our Profile matrix. We will be iterating through each kmer in
  every strand. Lets start with the first strand 'ACCGGTTGT'. Its first kmer is ACC which we add to Motifs.
  Now we our Profile function takes the profile of our Motifs string (which currently contains only ACC) and is inputed into 
  our function kmer_with_max_probability along with the next DNA string (GCTAGGTAC) with k = 3. 
  Remember this function iterates through GCTAGGTAC with a k length sliding window and returns the kmer with the highest prob 
  to be generated. For the sake of explanation we'll say the result is XYZ. This gets added to our Motifs list which is now
  [ACC, XYZ]. Now this gets repeated for all strings, and now the next iteration, our Profile(Motifs[0:j]) will have 2 motifs to
  create a profile out of.
  At the end of each iteration it checks the score of the current 'Motifs' list and compares it to our original Best Motifs and
  replaces it if its lower (because we want to minimize our score and find a more conservative strand).
  """ 
  
  best_score_motifs = []
  for i in range(len(Dna)):
      best_score_motifs.append(Dna[i][0:k])
  n = len(Dna[0])
  for i in range(n-k+1):
      Motifs = []
      Motifs.append(Dna[0][i:i+k])
      for j in range(1, len(Dna)):
          P = nuc_counter_profile(Motifs[0:j])
          Motifs.append(kmer_with_max_probability(Dna[j], k, P))
      if motif_scoring(Motifs) < motif_scoring(best_score_motifs):
          best_score_motifs = Motifs
  return best_score_motifs

In [0]:
print(Pr('TCGGG', nuc_counter_profile(sample_dna_list)))
print(kmer_with_max_probability(sample_genome, 3, nuc_counter_profile(sample_dna_list)))
print(GreedyMotifSearch(sample_dna_list, 3))

0.03662109375
TCG
['CGG', 'CGG', 'CGG', 'TGG']


Greedy algorithms traditionally in computer science fields are algorithms that try to find the 'best' or most optimal choice at every stage and iteration. Hence the name 'greedy'. Often times its synonomous with a brute force method as it involves looking at *every* option to decide if its better than the rest.
This greedy algorithm looks at every string and takes the kmer with the highest probability kmer (and therefore the most conserved) using what is at first a profile generated from a random kmer. However, with each addition of a new kmer to the motif matrix, it gets closer to a more conservative matrix.

##Gibbs Random Sampling & Monte Carlo Methods

In [0]:
def Normalize(Probabilities):
    '''
    Input = Probability dictionary
    Output = Dicitionary of normalized probabilities
    Divides every value in Probabilities by sum of all values in Probabilities. Handy probability tool to have.
    '''
    
    sum = 0
    for i in Probabilities.values():
        sum += i
    for j in Probabilities.keys():
        Probabilities[j] = Probabilities[j] / sum 
    return Probabilities


def motif_creation_double_matrix(Profile, Dna):
  '''
  Input: A pre-calculated profile matrix and a pre-calculated set of DNA strings
  Output: Motifs created via the'kmer_with_max_probability' function. Each one is made from one of the DNA strings via kmer_with_max_probability
  
  '''  
  k = len(Profile)
  return [kmer_with_max_probability(string, k, Profile) for string in Dna]

  
def rand_motif_from_DNAset(Dna, k):
  '''
  Input: Set of DNA strings and k describing length of kmer
  Ouput: List of kmers that are random selected from each one of the DNA strings
  
  '''
  random_kmers = []
  for i in range(len(Dna)):
      starter_number =  random.randint(0, len(Dna[0])-k)
      kmer = Dna[i][starter_number:starter_number+k]
      random_kmers.append(kmer)    
  return random_kmers
  
  
def random_greedy_algorithm(Dna, k):
  '''
  Input: Set of DNA strings and k describing length of kmer
  Output: The best scored motifs from the DNA strand
  
  Some similarities with our greedy algorithm. If we were to take for example the following set of DNA strings and k = 3 as input:
  ACCGGT
  GCTAGC
  CCTAGA
  
  We would take a random 3mer from each row with our random motif function so that we have for example:
  CCG
  AGC
  TAG
  
  We then create a profile matrix out of this. Now we have our original Dna input and a profile matrix. We can use our motif creation function again!
  Then we score the new one vs the old one.
  The key point here is that you can iterate over and over and over as much as you like given a DNA set of strings to gradually converge to the best scores.
  
  '''
  cur = rand_motif_from_DNAset(Dna, k)
  best_score_motifs = cur
  while True:
    Profile = nuc_counter_profile(cur)
    cur = motif_creation_double_matrix(Profile, Dna)
    if motif_scoring(cur) < motif_scoring(best_score_motifs):
      best_score_motifs = cur
    else:
      return best_score_motifs 
    
def random_outcome_generator(Probabilities):
    total = float(sum(Probabilities))
    r = random.random()
    partialSum = 0.0
    for i in range(len(Probabilities)):
        partialSum += Probabilities[i]
        if partialSum/total >= r:
            return i
    return -1
    
        
def random_dice(Probabilities):
  '''
  
  Input: A set of probabilities
  Output: Random outcome
  
  Essential a i-sided die where i is the number of possibilities there are. Useful for generating a random outcome.
  
  '''
  random_roll = random.uniform(0, 1)
  for key in Probabilities:
      random_roll -= Probabilities[key]
      if random_roll <= 0:
        return key

        
def random_kmer_from_string(string, profile, k):
  
  '''
  
  Input: string of DNA, pre-made profile and kmer length (k)
  Output: A random outcome of a kmer from the string
  
  The functionality is actually simpler than it seems. Given a string we are getting a random kmer from it. 
  Probabilities are based on a previously made profile
  
  '''

  t = len(string)
  probabilities = {}
  for i in range(0,t-k+1):
      probabilities[string[i:i+k]] = Pr(string[i:i+k], profile)
      probabilities = Normalize(probabilities)
  return random_dice(probabilities)


def gibbs_sampling(Dna, k, N ):
  
  '''
  Input: Set of DNA strings, k mer length, and N steps for iteration
  Output: Best scored motifs with their score from the random sampling
  
  This function is slightly different from our random_greedy_algorithm and its nuances are easier explained part by part as a comparison to that function. Shown below.
  '''
  
  t = len(Dna)
  cur = rand_motif_from_DNAset(Dna, k)
  best_score_motifs = cur
  best_score = motif_scoring(best_score_motifs) # Much like our greedy algorithm and random greedy algorithm, we start by filling up a base set of motifs to call best and compare to. These are randomly selected.
  
  for iteration in range(N): #We want to iterate this N times so we can gradually approach a good approximation and achieve convergence.
    
    i = random.randint(0, t-1) #Randomly generate a int in the range of our Dna length. The ith string will be our randomly removed and replaced string.
    Profile = nuc_counter_profile(cur[:i] + cur[i+1:]) #Gibbs sampling relies on removing a random string from the list to be ignored or used at a later iteration. So we create a profile with all but the ith string.
    
    Probabilities = [] #Records all the probabilities of our previously made profile.
    for j in range(len(Dna[i])-k+1):
      Probabilities.append(Pr(Dna[i][j:j+k], Profile))
      
    rand_roll = random_outcome_generator(Probabilities) #Our 'dice' now rolls itself based on our found probabilities and gives us a random outcome to use.
    cur[i] = Dna[i][rand_roll:rand_roll+k] #Our previously removed ith position is not replaced by
    cur_score = motif_scoring(cur) #From this point onwards its more or less the same with our previous algorithms. We score everything, compare it to the previous iterations score and replace it if its better.
    if cur_score < best_score: #This continues until improvement stops and we see our 'final' form.
      best_score_motifs = cur[:]
      best_score = cur_score
  return (best_score_motifs, best_score)

In [0]:
#A brief comparison of the two main methods here. Look! Their results are somewhat similar!
print(random_greedy_algorithm(sample_dna_list, 4))
print(gibbs_sampling(sample_dna_list, 4, 100))

['CGGG', 'CGGT', 'CGGG', 'TGGG']
(['CGGG', 'CCGG', 'CGGG', 'TGGG'], 2)


Theres two key main algorithms in this section, the rest of the functions simply helpy build them.
The first is random_greedy_algorithm. It is not quite as 'greedy' as our first greedy algorithm but it shares many similarities in its code and how it iterates. The main difference is where in the greedy algorithm we used the first kmer from each dna string as a starting motif matrix, here we are *randomly generating* our basis motif matrix. Since we can run this random algorithm as many times as we want, theres a chance of even hitting just one of the right kmers on to start. Which will gradually steer us on the right path towards convergence. However this is not optimal, and random_greedy_algorithm can replace many if not all strings in a motif iteration on one iteration, it may replace strings we want to keep.

Gibbs sampling on the other hand which is the 'capstone function' of this entire project, removes only ONE string with each iteration and can choose to toss it or keep it later on based on how the scoring performs. 
Gibbs sampling starts off the same as random_greedy_algorithm by selecting its first few kmers randomly but makes two more random choices in each iteration. It randomly selects an integer in the range of our DNA length and uses that as a cursor to randomly select a kmer to remove from our motifs. So instead of creating more motifs via the new profile and same DNA strand (motif_creation_double_matrix function) and constantly plugging the new result into that same function to generate an 'improved' matrix, it randomly selects things to generate a new motif in hopes that it is better. 

**Issues/Evaluation:** The advantages of these Monte Carlo randomized algorithms is that while it may not give us the exact solutions, they can be run many times to quickly help us reach an approximate one. However if you want an analytic solution this can be a problem. 
Not only this, it can converge on a local optimum, which is essnetially a set of solutions that is optimal with a specific set of data. As opposed to a global optimum which can be generalized to a wider range of contexts. High generalization is especially important for machine learning models.